In [1]:
from pyiron_base import TemplateJob, DataContainer
from pyiron_atomistics import Project
from pyiron_atomistics.atomistics.structure.has_structure import HasStructure
import os.path

In [2]:
##### creates a .xyz file from the IMD file in same name as the IMD file. the created .xyz file is in the 
###   same format as mentioned in the link : https://www.cgl.ucsf.edu/chimera/docs/UsersGuide/xyz.html 


class xyzgenerator:  
    def __init__(self, file_path, element):
        
        self.file_path = file_path  ##path of the IMD file
        
        self.filename = self.file_path.split('/')[-1].split('.')[0]+'.'+'xyz'  ##creating the name for the .xyz file from the path
        
        self.base_path = "/".join(self.file_path.split('/')[0:-1])  ## main base path where the param files, IMD files are kept and the .xyz file will be created
        
        self.element = element  ##symbol of the element used
        
        self.count = 0              ##saves the number of lines the header has except the last header line '#E\n'
        
        self.my_list = []           ##contains all the content of the IMD file after reading it
        
        self.new_list = []          ##contains all the lines of the IMD file expect the header part
        
        self.my_new_list =[]     ##contains only the x,y,z coordinates and along with the total number of entries at the top 
                                 ##replicating the .xyz format
            
        self.flag = True         ##this boolean indicates whether there are any columns for atom velocities and EPot or not
    
    def generator(self):
        
        FileHandler = open(self.file_path,"r")
        self.my_list.append(FileHandler.readlines())
        self.my_list = self.my_list[0]
        
        if int(self.my_list[0].split()[-1])==0 and int(self.my_list[0].split()[-2])==0:
            print('there is no columns for the atom velocities and no columns for the atom data, e.g. Epot')
            self.flag = True
    
        for i in range(0,len(self.my_list)):
            
            if self.my_list[i] == '#E\n' or self.my_list[i] == '#E \n' or self.my_list[i] == '#E':
                
                break
            else:
                
                self.count = self.count+1
                #self.new_list.append(self.my_list[i])
        else:
            print('there are columns for the atom velocities or columns for the atom data, e.g. Epot or both')
            self.flag = False
    
        self.new_list = self.my_list[self.count+1 :]
        
        
        #self.element = input("Element used : ")
        #print(self.element)
        
        #my_new_new_list = []
        while self.new_list[0] == '\n' or self.new_list[0] == '#\n':
            
            self.new_list.pop(0)
        
        if isinstance(int(self.new_list[0].split()[0]), int) == True:
            
            for i in range(0, len(self.new_list)):
                
                if self.flag == True:
                    
                    self.my_new_list.append(self.element+" "+" ".join(self.new_list[i].split()[-3 :])+" "+'\n')
            
            #my_new_new_list.append
            #" ".join(my_new_new_list[0])
        
        self.my_new_list.insert(0, str(len(self.new_list))+" "+'\n')
        
        self.my_new_list.insert(1, '##'+" "+'\n')
        
        return self.my_new_list
    
    def save_xyz(self):
        
        #filename = self.file_path.split('/')[-1].split('.')[0]+'.'+'xyz'
        
        list_temp = self.generator()
        
        #with open(os.path.join("/".join(self.file_path.split('/')[0:-1]), '{}'.format(filename)), 'w') as f:
        with open(os.path.join(self.base_path, '{}'.format(self.filename)), 'w') as f:
            #print(f)
            
            for i in list_temp:
            
                f.write(i)

In [4]:
class LEGO(TemplateJob, HasStructure):
    def __init__(self, project=None, job_name=None):
        super().__init__(project=project, job_name=job_name)
        # self.input is predefined by the TemplateJob and will be stored automatically
        self.input.update({
            # add the default values from ex.conf parameter file here
            'structure': 'fcc',
            'filename': 'output.xyz',
        })
        #self.input = {}
        # placeholder shell script, modify the script to point to your local LEGO installation
        # you might need to update this to the absolute path where you put the lego.sh file
        # later on this shell script will be provided via the pyiron_resources and this line 
        # will then be unnecessary
        #self.working_directory = os.path.abspath(r'/root/LEGO/1_CRYSTAL_GENERATION/1_CRYSTAL_GENERATION/')
        #self.executable = os.path.join(os.getcwd(), 'lego.sh')
        self.executable = os.path.join(r'/root/LEGO/1_CRYSTAL_GENERATION/1_CRYSTAL_GENERATION', 'lego.sh')
        self.mylist = []
        self.para = []
        #self.mydict = {}
        self.read_input()
#         self.write_input()
        #self.collect_output()
#         self._number_of_structures()
        #self._get_structure()
        #self.output_path = ''
    # convenience function for users with old parameter files#
#     def input_parameters(self):
#         para = input("Enter LEGO parameter's path:")
#         self.para.append(para)
#         return self.para[0]
    
    def read_input(self, filename = r'/root/LEGO/1_CRYSTAL_GENERATION/1_CRYSTAL_GENERATION/ex.confparam'):
        temp_list = []
        values = []
        keys = []
        #myDict = {}
        #with open(os.path.join(r'/root/LEGO/1_CRYSTAL_GENERATION/1_CRYSTAL_GENERATION', 'ex.confparam'), 'r') as f:
        with open(filename) as f:
            for a in f:
                temp_list.append(a)
                #print(os.getcwd())
                #print(temp_list)
                if temp_list[0]== '\n' or list(temp_list[0])[0] == '#':
                    temp_list = []
                else:
                    self.mylist.append(a)
                    temp_list = []
        
        for i in self.mylist:
            #print(i.split())
            #myDict[i.split()[0]] = i.split()[1]
            keys.append(i.split()[0])
            values.append(i.split()[1:])
    
        for i in range(0,len(keys)):
            
            self.input[keys[i]] = ' '.join(values[i])
        
#         if self.status.finished:
#             return 1
#         else:
#             return 0
        #self.input
            
            # this should read in the key value pairs from the given file and put them into self.input
    
    # necessary overload for TemplateJob
    def write_input(self):
        with open(os.path.join(r'/root/LEGO/1_CRYSTAL_GENERATION/1_CRYSTAL_GENERATION', 'lego.input'), 'w') as f:
            for k, v in self.input.items():
                # check with the lego input format whether this is correct
                f.write(f'{k} {v}\n')
                
    # necessary overload for TemplateJob
    def collect_output(self):
        #output_path = os.path.join(self.working_directory, self.input.outfile)
        #output_path = os.path.join(r'/root/LEGO/1_CRYSTAL_GENERATION/1_CRYSTAL_GENERATION', '{}'.format(self.input.filename))
        generator  = xyzgenerator(r'/root/LEGO/1_CRYSTAL_GENERATION/1_CRYSTAL_GENERATION/Ni_x101_z1-1-1.fcc', 'Al')
        generator.save_xyz()
        output_path = os.path.join(generator.base_path, '{}'.format(generator.filename))
        print(output_path)
        self.output.structure = self.project.create.structure.read(output_path)
        self.to_hdf()
        
    # necessary overload for HasStructure
    def _number_of_structures(self):
        if self.status.finished:
            return 1
        else:
            return 0
        
    # necessary overload for HasStructure
    def _get_structure(self, frame=-1, wrap_atoms=True):
        return self.output.structure

In [5]:
pr = Project('test148')
pr.remove_jobs_silently(recursive=True)
job = pr.create_job(job_type=LEGO, job_name="LEGOjob", delete_existing_job=True)

/tmp/ipykernel_18206/1839307524.py:2: DeprecationWarning: pyiron_base.project.generic.remove_jobs_silently is deprecated: Use pr.remove_jobs(silently=True) rather than pr.remove_jobs_silently()..
  pr.remove_jobs_silently(recursive=True)


In [6]:
job.run()

The job LEGOjob was saved and received the ID: 21
there is no columns for the atom velocities and no columns for the atom data, e.g. Epot
/root/LEGO/1_CRYSTAL_GENERATION/1_CRYSTAL_GENERATION/Ni_x101_z1-1-1.xyz


In [7]:
job.get_structure().plot3d()

NGLWidget()